Restrucutred version of the same named script in the `scripts` directory. Used to understand the code better and analysze whats going on.

In [1]:
import copy
from typing import Tuple

import trimesh

import scipy.spatial.transform
import numpy as np
from se3dif.datasets import AcronymGraspsDirectory
from se3dif.models.loader import load_model
from se3dif.samplers import ApproximatedGrasp_AnnealedLD, Grasp_AnnealedLD
from se3dif.utils import to_numpy, to_torch
from se3dif.visualization import create_gripper_marker

/home/moritz/miniconda3/envs/alr/envs/se3dif_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
N_GRASPS = 10               # argument values from README: 10,10,10. default in argparse: 200
OBJ_ID = 0                  # argument values from README: 0,10,12.  default in argparse: 0
OBJ_CLASS = "ScrewDriver"   # argument values from README: 'ScrewDriver','grasp_dif_mugs','Mug'. default in argparse: "Laptop"
N_ENVS = 30                 # hardcoded in the code with default value 30 in __main__
DEVICE = "cuda:0"           # argumant values from README: n.a., n.a., n.a. default in argparse: "cuda:0"
EVAL_SIM = False            # argument values from README: n.a., n.a., n.a. default in argparse: False
MODEL = "grasp_dif_multi"   # argument values from README: n.a., 'grasp_dif_mugs', n.a. default in argparse: 'grasp_dif_multi'
BATCH = 10                  # hardcoded in code with default value 10 in get_approximated_grasp_diffusion_field

In [3]:
def get_object_data(
    obj_id: int, obj_class: str
) -> Tuple["NpArray[3, N]", trimesh.Trimesh, "NpArray[4, 4]", "NpArray[4, 4]"]:
    """Load the mesh of the specified object and sample a pointcloud from it.
    The pointcloud and the mesh is rotated randomly, scaled by 8 and made zero mean.

    Args:
        obj_id (int): The id of the object to load.
        obj_class (str): The class of the object to load.

    Returns:
        Tuple["NpArray[3, N]", trimesh.Trimesh, "NpArray[4, 4]", "NpArray[4]"]: The pointcloud, 
            the mesh, the homogenaous transformation matrix of the rotation and the homogeneous
            transformation matrix of the translation.
    """
    
    # get mesh of object and sample pointcloud
    acronym_grasps = AcronymGraspsDirectory(data_type=obj_class)
    mesh = acronym_grasps.avail_obj[obj_id].load_mesh()
    pointcloud = mesh.sample(1000)

    # get a random rotation
    H_rot = np.eye(4)
    H_rot[:3, :3] = scipy.spatial.transform.Rotation.random().as_matrix()

    # apply the rotation to the pointcloud and mesh
    pointcloud = np.einsum("mn,bn->bm", H_rot[:3,:3], pointcloud)
    mesh.apply_transform(H_rot)

    # scale the pointcloud and mesh
    pointcloud *= 8.0
    mesh.apply_scale(8.0)
    
    # get the translation to make the pointcloud and mesh zero mean
    H_trans = np.eye(4)
    H_trans[:3, -1] = -np.mean(pointcloud, 0)
    pointcloud += H_trans[:3, -1]
    mesh.apply_transform(H_trans)

    return pointcloud, mesh, H_rot, H_trans


In [4]:
def get_fitted_grasp_generator(pointcloud, model):
    model.set_latent(to_torch(pointcloud[None, ...], DEVICE), batch=BATCH)
    generator = Grasp_AnnealedLD(
        model, batch=BATCH, T=70, T_fit=50, k_steps=2, device=DEVICE
    )
    return generator

In [5]:
pointcloud, mesh, H_rot, H_trans = get_object_data(OBJ_ID, OBJ_CLASS)
model = load_model({"device": DEVICE, "pretrained_model": MODEL})
generator = get_fitted_grasp_generator(pointcloud, model)
H_grasps = to_numpy(generator.sample()) # torch.Size([10, 4, 4])

In [6]:
print(pointcloud.min(0))
print(pointcloud.max(0))

[-0.50581346 -0.10307309 -0.31358986]
[0.26602659 0.10532769 0.62194449]


In [7]:
def visualization(mesh, H_grasps, scale):
    mesh = copy.deepcopy(mesh)
    H_grasps = copy.deepcopy(H_grasps)

    radius = 0.01
    axis_length = 0.5

    if scale:
        H_grasps[..., :3, -1] *=1/8.
        mesh = mesh.apply_scale(1/8)
        radius *= 1/8.
        axis_length *= 1/8.

    scene = trimesh.Scene(mesh)
    scene.add_geometry(trimesh.creation.axis(origin_size=radius, axis_length=axis_length))
    gripper_marker = create_gripper_marker()
    for H_grasp in H_grasps:
        scene.add_geometry(gripper_marker.copy().apply_transform(H_grasp))
    for H_grasp in H_grasps:
        scene.add_geometry(trimesh.primitives.Sphere(radius=0.01, center=H_grasp[:3, -1]))
    return scene
    

In [8]:
scene = visualization(mesh, H_grasps, scale=True)
scene.show()

In [9]:
scene = visualization(mesh, H_grasps, scale=False)
scene.show()

In [10]:
# H_grasp_scaled = copy.deepcopy(H_grasp)
# pointcloud_scaled = copy.deepcopy(pointcloud)
# mesh_scaled = copy.deepcopy(mesh)

# # H_grasp_scaled[:, :3, -1] = H_grasp_scaled[:, :3, -1] - H_trans[:3,-1]
# # H_grasp_scaled[..., :3, -1] *=1/8.
# # pointcloud_scaled *= 1/8.
# # mesh_scaled.apply_scale(1/8.)

# scene = trimesh.Scene()
# scene.add_geometry(trimesh.creation.axis(axis_length=1, origin_size=0.01, axis_radius=0.001))
# scene.add_geometry(mesh_scaled)
# for i in range(pointcloud_scaled.shape[0]):
#     point_sphere = trimesh.primitives.Sphere(radius=0.005, center=pointcloud_scaled[i])
#     point_sphere.visual.face_colors = [0,0,0,255]
#     scene.add_geometry(point_sphere)
# for i in range(H_grasp_scaled.shape[0]):
#     grasp_point_sphere = trimesh.primitives.Sphere(radius=0.01, center=H_grasp_scaled[i, :3, -1])
#     grasp_point_sphere.visual.face_colors = [255,0,0,255]
#     scene.add_geometry(grasp_point_sphere)
#     grasp_vector_cylinder = trimesh.creation.cylinder(radius=0.005, height=1, transform=H_grasp_scaled[i])
#     grasp_vector_cylinder.visual.face_colors = [255,0,0,255]
#     scene.add_geometry(grasp_vector_cylinder)
# scene.show()

In [11]:
# if (EVAL_SIM):
#     ## Evaluate Grasps in Simulation##
#     num_eval_envs = 10
#     evaluator = GraspSuccessEvaluator(OBJ_CLASS, n_envs=num_eval_envs, idxs=[args.obj_id] * num_eval_envs, viewer=True, device=DEVICE, \
#                                         rotations=[rot_quad]*num_eval_envs, enable_rel_trafo=False)
#     succes_rate = evaluator.eval_set_of_grasps(H_grasp)
#     print('Success cases : {}'.format(succes_rate))